In [1]:
# read it in to inspect it
with open('data/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
len(text)

1115389

In [3]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115389]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [6]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[59, 57,  1, 58, 56, 39, 47, 58],
        [30, 10,  0, 15, 53, 51, 43,  6],
        [34, 21, 26, 15, 17, 26, 32, 21],
        [27, 10,  0, 32, 46, 53, 59,  1]])
targets:
torch.Size([4, 8])
tensor([[57,  1, 58, 56, 39, 47, 58, 53],
        [10,  0, 15, 53, 51, 43,  6,  1],
        [21, 26, 15, 17, 26, 32, 21, 27],
        [10,  0, 32, 46, 53, 59,  1, 42]])
----
when input is [59] the target: 57
when input is [59, 57] the target: 1
when input is [59, 57, 1] the target: 58
when input is [59, 57, 1, 58] the target: 56
when input is [59, 57, 1, 58, 56] the target: 39
when input is [59, 57, 1, 58, 56, 39] the target: 47
when input is [59, 57, 1, 58, 56, 39, 47] the target: 58
when input is [59, 57, 1, 58, 56, 39, 47, 58] the target: 53
when input is [30] the target: 10
when input is [30, 10] the target: 0
when input is [30, 10, 0] the target: 15
when input is [30, 10, 0, 15] the target: 53
when input is [30, 10, 0, 15, 53] the target: 51
when input is [30, 

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None # logits would remain as (B,T,C)
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # Tensor with mean loss

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.5242, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [10]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [11]:
batch_size = 32
for steps in range(10_000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.470813274383545


In [12]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


llo br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;

Whe, oraingofof win!
RIfans picspeserer hee anf,
TOFonk? me ain ckntoty dedo bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
By bre ndy; by s afreanoo adicererupa anse tecorro llaus a!
OLengerithesinthengove fal ames trr
TI ar I t, mes, n sar; my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helour
Jginte the?
Thak orblyor id oree chot, p,
Beay Yolde Th ll


# Self-attention

In [14]:
B, T, C = 4, 8, 2   # batch, content, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [47]:
# v1
xbow = torch.zeros(x.shape)
for b in range(B):
    for t in range(T):
        xprev_mean = x[b, :t+1].mean(dim=0) # (t, C) -> (C,)
        xbow[b, t] = xprev_mean
xbow[0]

tensor([[-1.2630, -1.2298],
        [-1.3263, -0.7032],
        [-0.7793, -0.2108],
        [-0.8253,  0.0097],
        [-0.8572,  0.1852],
        [-0.7105,  0.2665],
        [-0.5849,  0.2817],
        [-0.3849,  0.1331]])

In [48]:
# v2, tril
wei = torch.tril(torch.ones((T, T)))
wei /= wei.sum(dim=-1, keepdim=True) # (T, T)
xbow2 = wei @ x   # (T, T) @ (B, T, C) -> (B, T, C)
xbow2[0]

tensor([[-1.2630, -1.2298],
        [-1.3263, -0.7032],
        [-0.7793, -0.2108],
        [-0.8253,  0.0097],
        [-0.8572,  0.1852],
        [-0.7105,  0.2665],
        [-0.5849,  0.2817],
        [-0.3849,  0.1331]])

In [50]:
# v3, tril + softmax
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

xbow3 = wei @ x
xbow3[0]

tensor([[-1.2630, -1.2298],
        [-1.3263, -0.7032],
        [-0.7793, -0.2108],
        [-0.8253,  0.0097],
        [-0.8572,  0.1852],
        [-0.7105,  0.2665],
        [-0.5849,  0.2817],
        [-0.3849,  0.1331]])

In [86]:
# v4, self-attention

head_size = 16
key = nn.Linear(C, head_size) # apply to each token independently C -> head_size
query = nn.Linear(C, head_size) # apply to each token independently C -> head_size
value = nn.Linear(C, head_size) # apply to each token independently C -> head_size

k = key(x) # (B, T, C) -> (B, T, head_size)
q = query(x) # (B, T, C) -> (B, T, head_size)
v = value(x) # (B, T, C) -> (B, T, head_size)

k_transp = k.transpose(1, 2) # (B, head_size, T)
wei = q @ k_transp # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

tril = torch.tril(torch.ones((T, T)))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei = wei / C**0.5   # scale by sqrt(C), so that variance is ~1

out = wei @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
# wei mixes rows of x (tokens) with previous tokens of x
# according to the similarity of their keys and queries

out.shape

torch.Size([4, 8, 16])

In [87]:
out[0]

tensor([[-0.6977, -0.2740,  0.3911,  0.0471,  0.5345, -0.9069,  0.7361,  0.2152,
          0.1282,  1.0211,  0.4593,  0.5926,  0.2504,  0.3334, -0.3364,  0.1112],
        [-0.4187, -0.6569,  0.3261, -0.1719,  0.3471, -0.4142,  0.5462, -0.3948,
          0.0902,  0.6141,  0.0657,  0.2975,  0.3421,  0.5031, -0.4993, -0.4318],
        [-0.5472, -0.3811,  0.3608, -0.2665,  0.3358, -0.1544,  0.5899, -0.2358,
          0.0789,  0.3445,  0.1686,  0.0839,  0.0828,  0.3574, -0.1609, -0.0645],
        [-0.3994, -0.4110,  0.3346, -0.7218,  0.0672,  0.9512,  0.4136, -0.7702,
          0.0088, -0.6646, -0.1757, -0.6790, -0.2457,  0.3300,  0.2102, -0.1483],
        [-0.3213, -0.5749,  0.3137, -0.6719,  0.0703,  0.8122,  0.3853, -0.8717,
          0.0145, -0.5184, -0.2413, -0.5627, -0.0964,  0.4160,  0.0146, -0.3672],
        [-0.4029, -0.4939,  0.3313, -0.5472,  0.1554,  0.5171,  0.4543, -0.6556,
          0.0346, -0.2575, -0.1020, -0.3679, -0.0558,  0.3873, -0.0196, -0.2449],
        [-0.4488, -0.3

In [84]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5673, 0.4327, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2111, 0.3748, 0.4140, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3017, 0.3363, 0.1066, 0.2555, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2143, 0.2516, 0.1034, 0.2121, 0.2186, 0.0000, 0.0000, 0.0000],
        [0.1131, 0.1768, 0.1330, 0.2074, 0.2269, 0.1429, 0.0000, 0.0000],
        [0.1167, 0.1765, 0.1023, 0.1886, 0.2048, 0.1147, 0.0964, 0.0000],
        [0.1672, 0.2375, 0.0554, 0.1883, 0.2014, 0.0721, 0.0586, 0.0195]],
       grad_fn=<SelectBackward0>)